In [1]:
import os
import glob
import shutil
import tifffile as tf
import matplotlib.pyplot as plt
# import cv2
import re
import numpy as np
from path import Path
from tqdm import tqdm

from aicsimageio.writers import OmeTiffWriter

In [2]:
data_root = "/home/kkobayas/koseki-data/Raman/data/HCR/"
save_root = "/home/kkobayas/koseki-data/Raman/data/unet/fovs"

In [3]:
def get_short_channels(c):
    if c=="647Nanog_546Col1a1_488Utf1":
        cs = "Nanog_Col1a1_Utf1"
    elif c=="647Krt7_564Peg10_488Bgn":
        cs = "Krt7_Peg10_Bgn"
    elif c=="647Fabp1_546Epcam_488Nnat":
        cs = "Fabp7_Epcam_Nnat"
    return cs

In [4]:
import javabridge as jb
def init_logger():
    """This is so that Javabridge doesn't spill out a lot of DEBUG messages
    during runtime.
    From CellProfiler/python-bioformats.
    """
    rootLoggerName = jb.get_static_field("org/slf4j/Logger",
                                         "ROOT_LOGGER_NAME",
                                         "Ljava/lang/String;")

    rootLogger = jb.static_call("org/slf4j/LoggerFactory",
                                "getLogger",
                                "(Ljava/lang/String;)Lorg/slf4j/Logger;",
                                rootLoggerName)

    logLevel = jb.get_static_field("ch/qos/logback/classic/Level",
                                   "WARN",
                                   "Lch/qos/logback/classic/Level;")

    jb.call(rootLogger,
            "setLevel",
            "(Lch/qos/logback/classic/Level;)V",
            logLevel)

In [5]:
import bioformats as bf
jb.start_vm(class_path=bf.JARS)
init_logger()

days = ["D8", "D8.5", "D9", "D9.5", "D10", "D10.5", "D11", "D11.5", "D12", "D12.5", "D13", "D13.5", "D14", "D14.5"]
# rounds = ["647Nanog_546Col1a1_488Utf1", "647Krt7_564Peg10_488Bgn", "647Fabp1_546Epcam_488Nnat"]
rounds = ["647Fabp1_546Epcam_488Nnat"]

n_z = 11
n_c = 5
w = 2048
h = 2048
datatype = "uint16"

expression = "hcr_all_(.*)_F(?P<Pos>.*).ims"

In [10]:
# %%capture
for r in rounds:
# for r in rounds:
    for d in tqdm(days):
        folder_path = os.path.join(data_root, d, r)

#         print("day:"+d+" round:"+r)

        # search through files e.g. hcr_pos_all_Pos*.tif
        for root, dirs, files in os.walk(folder_path):
            for filename in files:
                
                p = re.compile(expression)
                
                m=p.match(filename)
                if m != None:
                    pos = m.group('Pos')
                    pos = "Pos"+str(int(pos))
                    if pos=="Pos20":
                        pos="Bg"
                    
                    file_path = os.path.join(folder_path, filename)
                    
                    # BF, 647, 564, 488, DAPI
                    with bf.ImageReader(file_path) as reader:
#                         reader = bf.ImageReader(path=file_path)                  

                        cs = get_short_channels(r)
                        fn = d+"_"+pos+"_"+cs+".ome.tiff"
                        save_f = os.path.join(save_root,fn)

                        # read tif z and c-stacks
                        # then write ome-tiffs
                        # STZCYX
                        img = np.zeros((n_c,n_z,w,h),dtype=datatype)

                        for z in range(n_z):
                            for c in range(n_c):
                                img[c,z,:,:] = reader.read(c=c,z=z, rescale=False)
                                # bf.write_image(save_f, im, datatype,c=c,z=z,t=0,size_c=n_c,size_z=n_z)

                    OmeTiffWriter.save(img, save_f, overwrite_file=True, dim_order='CZYX')
                    
#                     print(fn)
jb.kill_vm()

100%|███████████████████████████████████████████████████████████████████████████| 14/14 [2:00:44<00:00, 517.47s/it]
